#  GLOBAL SUPERSTORE - SQL DATABASE SETUP (ERD VERSION)

**Project:** Global Superstore 2016 SQL Analysis

**Version:** 3.0 - MATCHES ERD EXACTLY

**Date:** December 31, 2025

---

##  DATABASE SCHEMA:

**7 TABLES:**
1. **dim_ship_mode** (ship_mode_id PK)
2. **dim_geography** (geography_id PK)
3. **dim_customers** (customer_id PK)
4. **dim_products** (product_id PK)
5. **dim_order_date** (date_id PK)
6. **dim_ship_date** (ship_date_id PK)
7. **fact_orders** (row_id PK)

**Foreign Keys in fact_orders:**
- customer_id → dim_customers.customer_id
- product_id → dim_products.product_id
- ship_mode_id → dim_ship_mode.ship_mode_id
- geography_id → dim_geography.geography_id
- date_id → dim_order_date.date_id
- ship_date_id → dim_ship_date.ship_date_id

---

##  STEP 1: MOUNT GOOGLE DRIVE

In [1]:
from google.colab import drive
import os

print(" Mounting Google Drive...")
drive.mount('/content/drive')
print(" Google Drive mounted!")

 Mounting Google Drive...
Mounted at /content/drive
 Google Drive mounted!


In [2]:
# Navigate to project folder
PROJECT_FOLDER = '/content/drive/MyDrive/Superstore Project'

print(f" Navigating to: {PROJECT_FOLDER}")
os.chdir(PROJECT_FOLDER)
print(f" Current directory: {os.getcwd()}")

# Verify clean data file exists
if os.path.exists('superstore_clean_FINAL.csv'):
    size_mb = os.path.getsize('superstore_clean_FINAL.csv') / (1024*1024)
    print(f" Found: superstore_clean_FINAL.csv ({size_mb:.2f} MB)")
else:
    print(" ERROR: superstore_clean_FINAL.csv not found!")
    raise FileNotFoundError("Clean data file not found")

 Navigating to: /content/drive/MyDrive/Superstore Project
 Current directory: /content/drive/MyDrive/Superstore Project
 Found: superstore_clean_FINAL.csv (14.20 MB)


##  STEP 2: IMPORT LIBRARIES

In [3]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print(" Libraries imported")
print(f"   Pandas: {pd.__version__}")
print(f"   SQLite: {sqlite3.sqlite_version}")

 Libraries imported
   Pandas: 2.2.2
   SQLite: 3.37.2


##  STEP 3: LOAD CLEAN DATA

In [4]:
print("="*80)
print("LOADING CLEAN DATA")
print("="*80)

df_clean = pd.read_csv('superstore_clean_FINAL.csv')

# Convert dates
df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'])
df_clean['Ship Date'] = pd.to_datetime(df_clean['Ship Date'])

print(f"\n Loaded: {len(df_clean):,} rows")
print(f"   Columns: {len(df_clean.columns)}")
print(f"   Memory: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   Date range: {df_clean['Order Date'].min().date()} to {df_clean['Ship Date'].max().date()}")

LOADING CLEAN DATA

 Loaded: 49,366 rows
   Columns: 32
   Memory: 54.08 MB
   Date range: 2012-01-01 to 2016-01-07


##  STEP 4: CREATE NORMALIZED TABLES

### Creating 7 tables matching ERD exactly

In [5]:
print("="*80)
print("CREATING DIMENSION TABLES")
print("="*80)

# 1. DIM_SHIP_MODE (renamed from dim_shipping)
print("\n  Creating dim_ship_mode...")

dim_ship_mode = pd.DataFrame({
    'ship_mode_id': [1, 2, 3, 4],
    'ship_mode': ['First Class', 'Same Day', 'Second Class', 'Standard Class']
})

print(f"    {len(dim_ship_mode)} rows")
print(f"   Primary Key: ship_mode_id")
print(f"   Columns: {list(dim_ship_mode.columns)}")

CREATING DIMENSION TABLES

  Creating dim_ship_mode...
    4 rows
   Primary Key: ship_mode_id
   Columns: ['ship_mode_id', 'ship_mode']


In [6]:
# 2. DIM_GEOGRAPHY
print("\n  Creating dim_geography...")

geo_cols = ['Country', 'State', 'City', 'Postal Code', 'Region', 'Market']
dim_geography = df_clean[geo_cols].copy()
dim_geography.columns = ['country', 'state', 'city', 'postal_code', 'region', 'market']
dim_geography = dim_geography.drop_duplicates().reset_index(drop=True)
dim_geography.insert(0, 'geography_id', range(1, len(dim_geography) + 1))

print(f"    {len(dim_geography):,} rows")
print(f"   Primary Key: geography_id")
print(f"   Columns: {list(dim_geography.columns)}")


  Creating dim_geography...
    5,047 rows
   Primary Key: geography_id
   Columns: ['geography_id', 'country', 'state', 'city', 'postal_code', 'region', 'market']


In [7]:
# 3. DIM_CUSTOMERS (simplified - only 5 columns as per ERD)
print("\n  Creating dim_customers...")

cust_cols = ['Customer ID', 'Customer Name', 'Segment']
dim_customers = df_clean[cust_cols].copy()
dim_customers.columns = ['customer_id', 'customer_name', 'segment']
dim_customers = dim_customers.drop_duplicates(subset=['customer_id']).reset_index(drop=True)

# Add first and last order dates
customer_dates = df_clean.groupby('Customer ID').agg({
    'Order Date': ['min', 'max']
}).reset_index()
customer_dates.columns = ['customer_id', 'first_order_date', 'last_order_date']

dim_customers = dim_customers.merge(customer_dates, on='customer_id', how='left')

print(f"    {len(dim_customers):,} rows")
print(f"   Primary Key: customer_id")
print(f"   Columns: {list(dim_customers.columns)}")


  Creating dim_customers...
    17,995 rows
   Primary Key: customer_id
   Columns: ['customer_id', 'customer_name', 'segment', 'first_order_date', 'last_order_date']


In [8]:
# 4. DIM_PRODUCTS (simplified - only 4 columns as per ERD)
print("\n  Creating dim_products...")

prod_cols = ['Product ID', 'Product Name', 'Category', 'Sub-Category']
dim_products = df_clean[prod_cols].copy()
dim_products.columns = ['product_id', 'product_name', 'category', 'sub_category']
dim_products = dim_products.drop_duplicates(subset=['product_id']).reset_index(drop=True)

print(f"    {len(dim_products):,} rows")
print(f"   Primary Key: product_id")
print(f"   Columns: {list(dim_products.columns)}")


  Creating dim_products...
    3,784 rows
   Primary Key: product_id
   Columns: ['product_id', 'product_name', 'category', 'sub_category']


In [9]:
# 5. DIM_ORDER_DATE (simplified - only 6 columns as per ERD)
print("\n  Creating dim_order_date...")

# Get unique order dates
order_dates = df_clean['Order Date'].unique()
order_dates = pd.to_datetime(order_dates)
order_dates = sorted(order_dates)

dim_order_date = pd.DataFrame({
    'date_id': pd.to_datetime(order_dates).date,
    'year': pd.to_datetime(order_dates).year,
    'quarter': pd.to_datetime(order_dates).quarter,
    'month': pd.to_datetime(order_dates).month,
    'month_name': pd.to_datetime(order_dates).strftime('%B'),
    'week': pd.to_datetime(order_dates).isocalendar().week,
    'day_of_week': pd.to_datetime(order_dates).dayofweek + 1
})

print(f"    {len(dim_order_date):,} rows")
print(f"   Primary Key: date_id")
print(f"   Columns: {list(dim_order_date.columns)}")


  Creating dim_order_date...
    1,430 rows
   Primary Key: date_id
   Columns: ['date_id', 'year', 'quarter', 'month', 'month_name', 'week', 'day_of_week']


In [10]:
# 6. DIM_SHIP_DATE (simplified - only 6 columns as per ERD)
print("\n  Creating dim_ship_date...")

# Get unique ship dates
ship_dates = df_clean['Ship Date'].unique()
ship_dates = pd.to_datetime(ship_dates)
ship_dates = sorted(ship_dates)

dim_ship_date = pd.DataFrame({
    'ship_date_id': pd.to_datetime(ship_dates).date,
    'year': pd.to_datetime(ship_dates).year,
    'quarter': pd.to_datetime(ship_dates).quarter,
    'month': pd.to_datetime(ship_dates).month,
    'month_name': pd.to_datetime(ship_dates).strftime('%B'),
    'week': pd.to_datetime(ship_dates).isocalendar().week,
    'day_of_week': pd.to_datetime(ship_dates).dayofweek + 1
})

print(f"    {len(dim_ship_date):,} rows")
print(f"   Primary Key: ship_date_id")
print(f"   Columns: {list(dim_ship_date.columns)}")


  Creating dim_ship_date...
    1,464 rows
   Primary Key: ship_date_id
   Columns: ['ship_date_id', 'year', 'quarter', 'month', 'month_name', 'week', 'day_of_week']


In [11]:
# 7. FACT_ORDERS (14 columns as per ERD)
print("\n  Creating fact_orders...")

# Map Ship Mode to ID
ship_mode_map = dict(zip(dim_ship_mode['ship_mode'], dim_ship_mode['ship_mode_id']))
df_clean['ship_mode_id'] = df_clean['Ship Mode'].map(ship_mode_map)

# Map Geography to ID
geo_lookup = dim_geography.copy()
geo_lookup['key'] = (
    geo_lookup['country'].astype(str) + '|' +
    geo_lookup['state'].fillna('').astype(str) + '|' +
    geo_lookup['city'].fillna('').astype(str) + '|' +
    geo_lookup['postal_code'].fillna('').astype(str)
)
geo_dict = dict(zip(geo_lookup['key'], geo_lookup['geography_id']))

df_clean['geo_key'] = (
    df_clean['Country'].astype(str) + '|' +
    df_clean['State'].fillna('').astype(str) + '|' +
    df_clean['City'].fillna('').astype(str) + '|' +
    df_clean['Postal Code'].fillna('').astype(str)
)
df_clean['geography_id'] = df_clean['geo_key'].map(geo_dict)

# Create fact table - EXACTLY AS PER ERD
fact_orders = pd.DataFrame({
    'row_id': df_clean['Row ID'],
    'order_id': df_clean['Order ID'],
    'customer_id': df_clean['Customer ID'],
    'product_id': df_clean['Product ID'],
    'ship_mode_id': df_clean['ship_mode_id'],
    'geography_id': df_clean['geography_id'],
    'date_id': df_clean['Order Date'].dt.date,  # FK to dim_order_date
    'ship_date_id': df_clean['Ship Date'].dt.date,  # FK to dim_ship_date
    'sales': df_clean['Sales'],
    'quantity': df_clean['Quantity'],
    'discount': df_clean['Discount'],
    'profit': df_clean['Profit'],
    'shipping_cost': df_clean['Shipping Cost'],
    'order_priority': df_clean['Order Priority']
})

print(f"    {len(fact_orders):,} rows")
print(f"   Primary Key: row_id")
print(f"   Columns: {list(fact_orders.columns)}")
print("\n   Foreign Keys:")
print("      - customer_id → dim_customers.customer_id")
print("      - product_id → dim_products.product_id")
print("      - ship_mode_id → dim_ship_mode.ship_mode_id")
print("      - geography_id → dim_geography.geography_id")
print("      - date_id → dim_order_date.date_id")
print("      - ship_date_id → dim_ship_date.ship_date_id")
print("\n All 7 tables created matching ERD exactly!")


  Creating fact_orders...
    49,366 rows
   Primary Key: row_id
   Columns: ['row_id', 'order_id', 'customer_id', 'product_id', 'ship_mode_id', 'geography_id', 'date_id', 'ship_date_id', 'sales', 'quantity', 'discount', 'profit', 'shipping_cost', 'order_priority']

   Foreign Keys:
      - customer_id → dim_customers.customer_id
      - product_id → dim_products.product_id
      - ship_mode_id → dim_ship_mode.ship_mode_id
      - geography_id → dim_geography.geography_id
      - date_id → dim_order_date.date_id
      - ship_date_id → dim_ship_date.ship_date_id

 All 7 tables created matching ERD exactly!


##  STEP 5: EXPORT CSV FILES TO GOOGLE DRIVE

In [12]:
print("="*80)
print("EXPORTING CSV FILES")
print("="*80)

# Export all tables as CSV
tables_to_export = [
    ('dim_ship_mode', dim_ship_mode),
    ('dim_geography', dim_geography),
    ('dim_customers', dim_customers),
    ('dim_products', dim_products),
    ('dim_order_date', dim_order_date),
    ('dim_ship_date', dim_ship_date),
    ('fact_orders', fact_orders)
]

for table_name, df in tables_to_export:
    filename = f"{table_name}.csv"
    df.to_csv(filename, index=False)
    size_kb = os.path.getsize(filename) / 1024
    print(f" {filename:<25} {len(df):>8,} rows  {size_kb:>8,.1f} KB")

print("\n All CSV files saved to Google Drive!")
print(f"   Location: {os.getcwd()}")

EXPORTING CSV FILES
 dim_ship_mode.csv                4 rows       0.1 KB
 dim_geography.csv            5,047 rows     277.4 KB
 dim_customers.csv           17,995 rows     993.1 KB
 dim_products.csv             3,784 rows     258.0 KB
 dim_order_date.csv           1,430 rows      45.1 KB
 dim_ship_date.csv            1,464 rows      46.1 KB
 fact_orders.csv             49,366 rows   5,475.1 KB

 All CSV files saved to Google Drive!
   Location: /content/drive/MyDrive/Superstore Project


##  STEP 6: CREATE SQLITE DATABASE

In [13]:
print("="*80)
print("CREATING SQLITE DATABASE")
print("="*80)

DB_FILE = 'superstore.db'

# Remove old database
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print(f"   Removed old {DB_FILE}")

# Create connection
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

print(f"\n Database created: {DB_FILE}")

CREATING SQLITE DATABASE
   Removed old superstore.db

 Database created: superstore.db


In [14]:
# Create table schemas - EXACTLY MATCHING ERD
print("\n Creating table schemas matching ERD...")

# 1. dim_ship_mode (2 columns)
cursor.execute("""
    CREATE TABLE dim_ship_mode (
        ship_mode_id INTEGER PRIMARY KEY,
        ship_mode TEXT NOT NULL UNIQUE
    )
""")
print(" dim_ship_mode")

# 2. dim_geography (7 columns)
cursor.execute("""
    CREATE TABLE dim_geography (
        geography_id INTEGER PRIMARY KEY,
        country TEXT NOT NULL,
        state TEXT,
        city TEXT,
        postal_code TEXT,
        region TEXT NOT NULL,
        market TEXT NOT NULL
    )
""")
print(" dim_geography")

# 3. dim_customers (5 columns)
cursor.execute("""
    CREATE TABLE dim_customers (
        customer_id TEXT PRIMARY KEY,
        customer_name TEXT,
        segment TEXT NOT NULL,
        first_order_date DATE,
        last_order_date DATE
    )
""")
print(" dim_customers")

# 4. dim_products (4 columns)
cursor.execute("""
    CREATE TABLE dim_products (
        product_id TEXT PRIMARY KEY,
        product_name TEXT NOT NULL,
        category TEXT NOT NULL,
        sub_category TEXT NOT NULL
    )
""")
print(" dim_products")

# 5. dim_order_date (7 columns)
cursor.execute("""
    CREATE TABLE dim_order_date (
        date_id DATE PRIMARY KEY,
        year INTEGER NOT NULL,
        quarter INTEGER NOT NULL,
        month INTEGER NOT NULL,
        month_name TEXT NOT NULL,
        week INTEGER NOT NULL,
        day_of_week INTEGER NOT NULL
    )
""")
print(" dim_order_date")

# 6. dim_ship_date (7 columns)
cursor.execute("""
    CREATE TABLE dim_ship_date (
        ship_date_id DATE PRIMARY KEY,
        year INTEGER NOT NULL,
        quarter INTEGER NOT NULL,
        month INTEGER NOT NULL,
        month_name TEXT NOT NULL,
        week INTEGER NOT NULL,
        day_of_week INTEGER NOT NULL
    )
""")
print(" dim_ship_date")

# 7. fact_orders (14 columns with 6 FKs)
cursor.execute("""
    CREATE TABLE fact_orders (
        row_id INTEGER PRIMARY KEY,
        order_id TEXT NOT NULL,
        customer_id TEXT NOT NULL,
        product_id TEXT NOT NULL,
        ship_mode_id INTEGER NOT NULL,
        geography_id INTEGER NOT NULL,
        date_id DATE NOT NULL,
        ship_date_id DATE NOT NULL,
        sales REAL NOT NULL,
        quantity INTEGER NOT NULL,
        discount REAL NOT NULL,
        profit REAL NOT NULL,
        shipping_cost REAL NOT NULL,
        order_priority TEXT NOT NULL,

        FOREIGN KEY (customer_id) REFERENCES dim_customers(customer_id),
        FOREIGN KEY (product_id) REFERENCES dim_products(product_id),
        FOREIGN KEY (ship_mode_id) REFERENCES dim_ship_mode(ship_mode_id),
        FOREIGN KEY (geography_id) REFERENCES dim_geography(geography_id),
        FOREIGN KEY (date_id) REFERENCES dim_order_date(date_id),
        FOREIGN KEY (ship_date_id) REFERENCES dim_ship_date(ship_date_id)
    )
""")
print(" fact_orders")

conn.commit()
print("\n All 7 tables created - EXACT MATCH TO ERD!")


 Creating table schemas matching ERD...
 dim_ship_mode
 dim_geography
 dim_customers
 dim_products
 dim_order_date
 dim_ship_date
 fact_orders

 All 7 tables created - EXACT MATCH TO ERD!


In [15]:
# Load data into database
print("\n Loading data into database...")

dim_ship_mode.to_sql('dim_ship_mode', conn, if_exists='append', index=False)
print("    dim_ship_mode")

dim_geography.to_sql('dim_geography', conn, if_exists='append', index=False)
print("    dim_geography")

dim_customers.to_sql('dim_customers', conn, if_exists='append', index=False)
print("    dim_customers")

dim_products.to_sql('dim_products', conn, if_exists='append', index=False)
print("    dim_products")

dim_order_date.to_sql('dim_order_date', conn, if_exists='append', index=False)
print("    dim_order_date")

dim_ship_date.to_sql('dim_ship_date', conn, if_exists='append', index=False)
print("    dim_ship_date")

fact_orders.to_sql('fact_orders', conn, if_exists='append', index=False)
print("    fact_orders")

conn.commit()
print("\n All data loaded successfully!")


 Loading data into database...
    dim_ship_mode
    dim_geography
    dim_customers
    dim_products
    dim_order_date
    dim_ship_date
    fact_orders

 All data loaded successfully!


In [16]:
# Create indexes
print("\n Creating indexes...")

indexes = [
    "CREATE INDEX idx_orders_customer ON fact_orders(customer_id)",
    "CREATE INDEX idx_orders_product ON fact_orders(product_id)",
    "CREATE INDEX idx_orders_geography ON fact_orders(geography_id)",
    "CREATE INDEX idx_orders_date ON fact_orders(date_id)",
    "CREATE INDEX idx_orders_ship_date ON fact_orders(ship_date_id)",
    "CREATE INDEX idx_geography_country ON dim_geography(country)",
    "CREATE INDEX idx_geography_market ON dim_geography(market)",
    "CREATE INDEX idx_products_category ON dim_products(category)"
]

for idx in indexes:
    cursor.execute(idx)

conn.commit()
print(f" Created {len(indexes)} indexes")


 Creating indexes...
 Created 8 indexes


##  STEP 7: DATABASE STATISTICS

In [17]:
print("="*80)
print("DATABASE STATISTICS")
print("="*80)

# Table counts
tables = ['dim_ship_mode', 'dim_geography', 'dim_customers', 'dim_products',
          'dim_order_date', 'dim_ship_date', 'fact_orders']
print("\n TABLE ROW COUNTS:")
total = 0
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    total += count
    print(f"   {table:<20} {count:>10,} rows")
print(f"   {'─'*20} {'-'*12}")
print(f"   {'TOTAL':<20} {total:>10,} rows")

# Database size
db_size = os.path.getsize(DB_FILE) / (1024*1024)
print(f"\n DATABASE SIZE: {db_size:.2f} MB")

# Business metrics
query = "SELECT SUM(sales), SUM(profit), AVG(sales) FROM fact_orders"
cursor.execute(query)
revenue, profit, avg_order = cursor.fetchone()

print(f"\n BUSINESS METRICS:")
print(f"   Total Revenue:  ${revenue:>12,.2f}")
print(f"   Total Profit:   ${profit:>12,.2f}")
print(f"   Profit Margin:  {(profit/revenue*100):>12.2f}%")
print(f"   Avg Order:      ${avg_order:>12,.2f}")

DATABASE STATISTICS

 TABLE ROW COUNTS:
   dim_ship_mode                 4 rows
   dim_geography             5,047 rows
   dim_customers            17,995 rows
   dim_products              3,784 rows
   dim_order_date            1,430 rows
   dim_ship_date             1,464 rows
   fact_orders              49,366 rows
   ──────────────────── ------------
   TOTAL                    79,090 rows

 DATABASE SIZE: 13.40 MB

 BUSINESS METRICS:
   Total Revenue:  $12,152,049.84
   Total Profit:   $1,411,863.53
   Profit Margin:         11.62%
   Avg Order:      $      246.16


##  STEP 8: TEST QUERIES

In [18]:
# Test basic query
test_query = """
SELECT
    c.customer_name,
    p.product_name,
    g.country,
    f.sales,
    f.profit
FROM fact_orders f
JOIN dim_customers c ON f.customer_id = c.customer_id
JOIN dim_products p ON f.product_id = p.product_id
JOIN dim_geography g ON f.geography_id = g.geography_id
ORDER BY f.sales DESC
LIMIT 10
"""

print("="*80)
print("TEST QUERY 1: Top 10 Orders")
print("="*80)

result = pd.read_sql_query(test_query, conn)
print("\n", result.to_string(index=False))
print("\n Basic query works!")

TEST QUERY 1: Top 10 Orders

      customer_name                                          product_name       country    sales   profit
       Sean Miller Cisco TelePresence System EX90 Videoconferencing Unit United States 22638.48 -1811.08
      Tamara Chand                 Canon imageCLASS 2200 Advanced Copier United States 17499.95  8399.98
      Raymond Buch                 Canon imageCLASS 2200 Advanced Copier United States 13999.96  6719.98
      Tom Ashbrook                 Canon imageCLASS 2200 Advanced Copier United States 11199.97  3919.99
      Hunter Lopez                 Canon imageCLASS 2200 Advanced Copier United States 10499.97  5039.99
     Adrian Barton      GBC Ibimaster 500 Manual ProClick Binding System United States  9892.74  4946.37
      Sanjit Chand                  Ibico EPK-21 Electric Binding System United States  9449.95  4630.48
      Bill Shonely      3D Systems Cube Printer, 2nd Generation, Magenta United States  9099.93  2365.98
Christopher Conant       

In [19]:
# Test date dimension queries
date_query = """
SELECT
    d.year,
    d.quarter,
    ROUND(SUM(f.sales), 2) as total_revenue,
    COUNT(*) as total_orders
FROM fact_orders f
JOIN dim_order_date d ON f.date_id = d.date_id
GROUP BY d.year, d.quarter
ORDER BY d.year, d.quarter
"""

print("="*80)
print("TEST QUERY 2: Quarterly Sales Trends")
print("="*80)

result = pd.read_sql_query(date_query, conn)
print("\n", result.to_string(index=False))
print("\n Date joins work perfectly!")
print("   Foreign Key: fact_orders.date_id → dim_order_date.date_id")

TEST QUERY 2: Quarterly Sales Trends

  year  quarter  total_revenue  total_orders
 2012        1      326593.42          1308
 2012        2      453344.73          1952
 2012        3      587707.51          2343
 2012        4      793045.71          3040
 2013        1      384975.31          1563
 2013        2      604649.59          2538
 2013        3      721612.06          2864
 2013        4      883162.65          3583
 2014        1      531617.85          2015
 2014        2      809484.72          3257
 2014        3      889967.60          3879
 2014        4     1027315.85          4134
 2015        1      666838.34          2644
 2015        2      892610.39          3917
 2015        3     1161319.70          4615
 2015        4     1417804.41          5714

 Date joins work perfectly!
   Foreign Key: fact_orders.date_id → dim_order_date.date_id


---

#  SUCCESS! DATABASE MATCHES ERD EXACTLY!

---

##  DATABASE SCHEMA:

### **7 TABLES CREATED:**

1. **dim_ship_mode** (2 columns)
   - ship_mode_id, ship_mode

2. **dim_geography** (7 columns)
   - geography_id, country, state, city, postal_code, region, market

3. **dim_customers** (5 columns)
   - customer_id, customer_name, segment, first_order_date, last_order_date

4. **dim_products** (4 columns)
   - product_id, product_name, category, sub_category

5. **dim_order_date** (7 columns)
   - date_id, year, quarter, month, month_name, week, day_of_week

6. **dim_ship_date** (7 columns)
   - ship_date_id, year, quarter, month, month_name, week, day_of_week

7. **fact_orders** (14 columns)
   - row_id, order_id, customer_id, product_id, ship_mode_id, geography_id
   - date_id, ship_date_id, sales, quantity, discount, profit
   - shipping_cost, order_priority

---

##  FOREIGN KEY RELATIONSHIPS:

- fact_orders.customer_id → dim_customers.customer_id (1:N)
- fact_orders.product_id → dim_products.product_id (1:N)
- fact_orders.ship_mode_id → dim_ship_mode.ship_mode_id (1:N)
- fact_orders.geography_id → dim_geography.geography_id (1:N)
- fact_orders.date_id → dim_order_date.date_id (1:N)
- fact_orders.ship_date_id → dim_ship_date.ship_date_id (1:N)

---

##  FILES CREATED:

**In Google Drive:**
- dim_ship_mode.csv
- dim_geography.csv
- dim_customers.csv
- dim_products.csv
- dim_order_date.csv
- dim_ship_date.csv
- fact_orders.csv
- superstore.db

---

**Ready for SQL queries!**

---

##  YOUR SQL QUERIES

In [20]:
# Your SQL queries here
query = """
SELECT * FROM fact_orders LIMIT 5
"""

result = pd.read_sql_query(query, conn)
print(result)

   row_id                 order_id customer_id   product_id  ship_mode_id  \
0       1  MX-2015-SC2057582-42279  SC-2057582  OFF-LA-4658             4   
1       2  MX-2013-KW1657028-41562  KW-1657028  FUR-FU-6238             4   
2       3  MX-2013-KW1657028-41562  KW-1657028  FUR-BO-4845             4   
3       4  MX-2013-KW1657028-41562  KW-1657028  OFF-BI-3720             4   
4       5  MX-2013-KW1657028-41562  KW-1657028  OFF-AR-5905             4   

   geography_id     date_id ship_date_id   sales  quantity  discount  profit  \
0           188  2015-10-02   2015-10-06   13.08         3       0.0    4.56   
1           431  2013-10-15   2013-10-20  252.16         8       0.0   90.72   
2           431  2013-10-15   2013-10-20  193.28         2       0.0   54.08   
3          3183  2013-10-15   2013-10-20   35.44         4       0.0    4.96   
4           431  2013-10-15   2013-10-20   71.60         2       0.0   11.44   

   shipping_cost order_priority  
0           2.03      

In [21]:
# Your SQL queries here
query = """
SELECT * FROM dim_ship_date LIMIT 5
"""

result = pd.read_sql_query(query, conn)
print(result)

  ship_date_id  year  quarter  month month_name  week  day_of_week
0   2012-01-03  2012        1      1    January     1            2
1   2012-01-05  2012        1      1    January     1            4
2   2012-01-06  2012        1      1    January     1            5
3   2012-01-07  2012        1      1    January     1            6
4   2012-01-08  2012        1      1    January     1            7


In [22]:
# Your SQL queries here
query = """
SELECT * FROM dim_order_date LIMIT 5
"""

result = pd.read_sql_query(query, conn)
print(result)

      date_id  year  quarter  month month_name  week  day_of_week
0  2012-01-01  2012        1      1    January    52            7
1  2012-01-02  2012        1      1    January     1            1
2  2012-01-03  2012        1      1    January     1            2
3  2012-01-04  2012        1      1    January     1            3
4  2012-01-05  2012        1      1    January     1            4


---

# Business Intelligence Analysis

This analysis addresses five key business questions:

1. Overall Company Performance
2. Sales Trends Over Time
3. Product Category Profitability
4. Top Revenue-Generating Products
5. Market Performance Analysis

---

## Business Question 1: Overall Company Performance

**Objective:** Assess company-wide financial health and key performance indicators.

In [23]:
query = """
SELECT
    COUNT(*) as total_orders,
    COUNT(DISTINCT order_id) as unique_orders,
    ROUND(SUM(sales), 2) as total_revenue,
    ROUND(SUM(profit), 2) as total_profit,
    ROUND(SUM(profit) * 100.0 / SUM(sales), 2) as profit_margin_pct,
    ROUND(AVG(sales), 2) as avg_order_value,
    SUM(CASE WHEN profit > 0 THEN 1 ELSE 0 END) as profitable_orders,
    SUM(CASE WHEN profit < 0 THEN 1 ELSE 0 END) as unprofitable_orders,
    ROUND(SUM(CASE WHEN profit < 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as unprofitable_pct
FROM fact_orders;
"""

result = pd.read_sql_query(query, conn)

from IPython.display import display, HTML

report = result.T.reset_index()
report.columns = ['Metric', 'Value']

def format_values(row):
    val = row['Value']
    metric = row['Metric'].lower()
    if isinstance(val, (int, float)):
        if any(x in metric for x in ['pct', 'margin']):
            return f"{val:.2f}%"
        if any(x in metric for x in ['revenue', 'profit', 'value']):
            return f"${val:,.2f}"
        return f"{val:,.0f}"
    return val

report['Value'] = report.apply(format_values, axis=1)

styled_report = report.style.hide(axis='index') \
    .set_table_styles([
        {'selector': 'th', 'props': [
            ('background-color', '#2c3e50'),
            ('color', 'white'),
            ('font-family', 'Arial'),
            ('font-weight', 'bold'),
            ('text-align', 'left'),
            ('padding', '12px')
        ]},
        {'selector': 'td', 'props': [
            ('text-align', 'left'),
            ('padding', '10px'),
            ('border-bottom', '1px solid #ddd')
        ]}
    ]) \
    .set_properties(**{
        'background-color': '#ffffff',
        'color': '#2c3e50',
        'font-family': 'Arial'
    })

display(HTML("<h3 style='color: #2c3e50; font-family: Arial;'>Company Performance Metrics</h3>"))
display(styled_report)

Metric,Value
total_orders,"49,366"
unique_orders,"25,213"
total_revenue,"$12,152,049.84"
total_profit,"$1,411,863.53"
profit_margin_pct,11.62%
avg_order_value,$246.16
profitable_orders,"$36,642.00"
unprofitable_orders,"$12,083.00"
unprofitable_pct,24.48%


## Business Question 2: Sales Trends Over Time

**Objective:** Identify growth patterns and seasonal trends.

In [24]:
query = """
SELECT
    d.year,
    d.quarter,
    ROUND(SUM(f.sales), 2) as total_revenue,
    COUNT(*) as total_orders,
    ROUND(SUM(f.sales) / COUNT(*), 2) as avg_order_value,
    LAG(ROUND(SUM(f.sales), 2)) OVER (ORDER BY d.year, d.quarter) as previous_quarter_revenue,
    ROUND((SUM(f.sales) - LAG(SUM(f.sales)) OVER (ORDER BY d.year, d.quarter)) * 100.0 /
          LAG(SUM(f.sales)) OVER (ORDER BY d.year, d.quarter), 2) as growth_pct
FROM fact_orders f
JOIN dim_order_date d ON f.date_id = d.date_id
GROUP BY d.year, d.quarter
ORDER BY d.year, d.quarter;
"""

result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

 year  quarter  total_revenue  total_orders  avg_order_value  previous_quarter_revenue  growth_pct
 2012        1      326593.42          1308           249.69                       NaN         NaN
 2012        2      453344.73          1952           232.25                 326593.42       38.81
 2012        3      587707.51          2343           250.84                 453344.73       29.64
 2012        4      793045.71          3040           260.87                 587707.51       34.94
 2013        1      384975.31          1563           246.31                 793045.71      -51.46
 2013        2      604649.59          2538           238.24                 384975.31       57.06
 2013        3      721612.06          2864           251.96                 604649.59       19.34
 2013        4      883162.65          3583           246.49                 721612.06       22.39
 2014        1      531617.85          2015           263.83                 883162.65      -39.81
 2014     

In [25]:
query = """
SELECT
    month,
    month_name,
    ROUND(AVG(monthly_revenue), 2) as avg_monthly_revenue,
    ROUND(MIN(monthly_revenue), 2) as min_monthly_revenue,
    ROUND(MAX(monthly_revenue), 2) as max_monthly_revenue
FROM (
    SELECT
        d.year,
        d.month,
        d.month_name,
        SUM(f.sales) as monthly_revenue
    FROM fact_orders f
    JOIN dim_order_date d ON f.date_id = d.date_id
    GROUP BY d.year, d.month, d.month_name
) monthly
GROUP BY month, month_name
ORDER BY month;
"""

result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

 month month_name  avg_monthly_revenue  min_monthly_revenue  max_monthly_revenue
     1    January            162793.66             95639.73            232622.65
     2   February            130438.18             95589.27            178690.40
     3      March            184274.39            133335.03            255525.29
     4      April            166762.00            107552.45            229708.83
     5        May            218869.58            148261.38            277135.20
     6       June            304390.77            197530.90            386519.97
     7       July            180402.38            114918.52            245229.13
     8     August            315095.81            204702.50            444820.42
     9  September            344653.53            268086.49            471270.15
    10    October            280787.93            207972.68            403545.78
    11   November            372106.91            279309.07            532777.35
    12   December           

## Business Question 3: Product Category Profitability

**Objective:** Compare performance across Technology, Furniture, and Office Supplies.

In [26]:
query = """
SELECT
    p.category,
    ROUND(SUM(f.sales), 2) as total_revenue,
    ROUND(SUM(f.profit), 2) as total_profit,
    ROUND(SUM(f.profit) * 100.0 / SUM(f.sales), 2) as profit_margin_pct,
    COUNT(*) as order_count,
    ROUND(AVG(f.sales), 2) as avg_order_value,
    ROUND(SUM(f.sales) * 100.0 / (SELECT SUM(sales) FROM fact_orders), 2) as revenue_pct
FROM fact_orders f
JOIN dim_products p ON f.product_id = p.product_id
GROUP BY p.category
ORDER BY profit_margin_pct DESC;
"""

result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

       category  total_revenue  total_profit  profit_margin_pct  order_count  avg_order_value  revenue_pct
     Technology     4563335.99     640654.00              14.04         9754           467.84        37.55
Office Supplies     3649169.10     504439.87              13.82        30124           121.14        30.03
      Furniture     3939544.75     266769.66               6.77         9488           415.21        32.42


## Business Question 4: Top Revenue-Generating Products

**Objective:** Identify best-performing products and revenue concentration.

In [27]:
query = """
SELECT
    p.product_name,
    p.category,
    COUNT(*) as order_count,
    ROUND(SUM(f.sales), 2) as total_revenue,
    ROUND(SUM(f.sales) * 100.0 / (SELECT SUM(sales) FROM fact_orders), 2) as pct_of_total_revenue,
    ROUND(SUM(f.profit), 2) as total_profit,
    ROUND(SUM(f.profit) * 100.0 / SUM(f.sales), 2) as profit_margin_pct
FROM fact_orders f
JOIN dim_products p ON f.product_id = p.product_id
GROUP BY p.product_id, p.product_name, p.category
ORDER BY total_revenue DESC
LIMIT 10;
"""

result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

                                            product_name   category  order_count  total_revenue  pct_of_total_revenue  total_profit  profit_margin_pct
                            Apple Smart Phone, Full Size Technology           49       84548.45                  0.70       5826.54               6.89
                            Cisco Smart Phone, Full Size Technology           37       75138.75                  0.62      16951.96              22.56
                         Motorola Smart Phone, Full Size Technology           35       70605.70                  0.58      16594.13              23.50
                            Nokia Smart Phone, Full Size Technology           41       63834.03                  0.53       8528.04              13.36
                   Canon imageCLASS 2200 Advanced Copier Technology            5       61599.83                  0.51      25199.94              40.91
              Hon Executive Leather Armchair, Adjustable  Furniture           47       51544.9

## Business Question 5: Market Performance Analysis

**Objective:** Evaluate geographic market performance for strategic planning.

In [28]:
query = """
SELECT
    g.market,
    ROUND(SUM(f.sales), 2) as total_revenue,
    ROUND(SUM(f.profit), 2) as total_profit,
    ROUND(SUM(f.profit) * 100.0 / SUM(f.sales), 2) as profit_margin_pct,
    COUNT(*) as order_count,
    COUNT(DISTINCT f.customer_id) as unique_customers,
    ROUND(SUM(f.sales) / COUNT(DISTINCT f.customer_id), 2) as revenue_per_customer,
    ROUND(SUM(f.sales) * 100.0 / (SELECT SUM(sales) FROM fact_orders), 2) as revenue_pct
FROM fact_orders f
JOIN dim_geography g ON f.geography_id = g.geography_id
GROUP BY g.market
ORDER BY total_revenue DESC;
"""

result = pd.read_sql_query(query, conn)
print(result.to_string(index=False))

      market  total_revenue  total_profit  profit_margin_pct  order_count  unique_customers  revenue_per_customer  revenue_pct
Asia Pacific     3892235.99     388950.18               9.99        13750              5083                765.74        32.03
      Europe     3169334.42     436095.50              13.76        11320              4196                755.32        26.08
        USCA     2267664.03     295616.11              13.04         9970              2851                795.39        18.66
       LATAM     2074648.73     207135.62               9.98         9916              3790                547.40        17.07
      Africa      748166.67      84066.12              11.24         4410              2075                360.56         6.16


---

## Analysis Complete

**Five business questions addressed through SQL analysis:**

1. Overall Company Performance - Key financial metrics and order profitability
2. Sales Trends Over Time - Quarterly growth patterns and monthly seasonality
3. Product Category Profitability - Technology, Furniture, and Office Supplies comparison
4. Top Revenue-Generating Products - Top 10 products and revenue concentration
5. Market Performance Analysis - Geographic distribution across five markets

**Next Steps:**
- Results exported for dashboard creation in Tableau and Power BI
- Executive Dashboard (Power BI) - Strategic KPIs and trends
- Product Dashboard (Tableau) - Detailed product and category analysis

---

In [29]:
conn.close()
print("Analysis complete. Database connection closed.")

Analysis complete. Database connection closed.
